# Movie Recommendations using Collaborative Filtering
### - Dataset used: Group Lens 100K movie dataset

In [ ]:
#Imports
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import scipy.io
from scipy.optimize import fmin_cg

#CONSTANTS
"""Read data from files."""
DATAPATH = "/Users/debojitkaushik/collaborative-filtering/ml-100k/"
Y_cols  = ['user_id', 'item_id', 'rating', 'timestamp']
data = pd.read_csv(DATAPATH + 'u.data', sep = '\t', encoding = 'latin-1', names = Y_cols).drop("timestamp", 1)
data

### - Preprocessing/Formatting
Create movies VS user matrix and ratings matrix. Create Y matrix and ratings matrix. Data is a dataframe with UserID|Movie_id|Rating format. Need to Vectorize this into a ratings matrix of MovieID vs UserID. Create the R matrix, which will have 1 if a user has rated or 0 if not.

In [ ]:
nm, nu = len(data['item_id'].unique()), len(data['user_id'].unique())
nf = 10

# Initilize two matrices of size (nm, nu) and fill in the data.
# # **Slow method. Need to optimize this. Will look into Pandas methods more. 
Y, ratings  = np.zeros((nm, nu)), np.zeros((nm, nu))
for it, item in data.iterrows():
    ratings[item['item_id']-1][item['user_id']-1] = 1.0
    Y[item['item_id']-1][item['user_id']-1] = item['rating']
print("Y:" , Y.shape)
print("R:", ratings.shape)

#OPTIONAL DATA SOURCE:
# datafile = 'aux_data/ex8_movies.mat'
# mat = scipy.io.loadmat( datafile )
# Y = mat['Y']
# ratings = mat['R']

In [ ]:
#Visualize the data matrix.
fig = plt.figure(figsize=(6,6*(nm/nu)))
dummy = plt.imshow(Y)
dummy = plt.colorbar()
dummy = plt.ylabel('Movies (%d)'%nm,fontsize=20)
dummy = plt.xlabel('Users (%d)'%nu,fontsize=20)
#We can see the sparseness of the matrix as it is dominated by 0 values.

### - Cost Function and Gradient definitions

In [ ]:
#OPTIONAL: loading Theta and X matrices. 
# mat = scipy.io.loadmat("data/ex8_movieParams.mat")
# theta, X, nf = mat['Theta'], mat['X'], mat['num_features']

#Let's randomly initialize Theta and X matrices for starting points before we start learning.
X, Theta = np.random.rand(nm, nf), np.random.rand(nu, nf)
print("Shapes of Theta, X, Y, ratings:", [i.shape for i in [Theta, X, Y, ratings]])

In [ ]:
'''Method to flatten theta and X matrices into one vector'''
def flatten_mat(X, Theta):
    try:
        return np.concatenate((X.flatten(), Theta.flatten()))
    except Exception as e:
        print(e)
        
'''Method to extract X and theta from the flattened matrix and reshape.'''
def unflatten(flattened_mat, mynu, mynm, mynf):
    #X dimensions: nm*nf, theta dimensions: nu*nf
    new_X = flattened_mat[:int(nm*nf)].reshape(mynm, mynf)
    new_theta = flattened_mat[int(nm*nf):].reshape(mynu, mynf)
    return new_X, new_theta

'''Compute cost function'''
def CFCost(params, myY, myR, mynu, mynm, mynf, myLambda = 0):
    '''
    args:
        params: X+Theta (flattened vector)
        myY: Movies VS Users matrix.
        nu, nm, nf: No of user, no of movies, no of features
    '''
    #Extract X and theta from params which is a merged vector.
    myX, myTheta = unflatten(params, mynu, mynm, mynf)
    
    #1/2∑((theta).T * X - Y)^2 + Regularization term
    term1 = myX.dot(myTheta.T)
    term1 = np.multiply(term1, myR)
    cost = 0.5 * np.sum(np.square(term1 - myY))
    
    #Regularization factors. Twice, for lambda and for X.
    #Regularizer: (λ/2) * (theta)^2 or (X)^2
    cost += (myLambda/2) * (np.sum(np.square(myTheta)))
    cost += (myLambda/2) * (np.sum(np.square(myX)))
    return cost

print(CFCost(flatten_mat(X, Theta), Y, ratings, nu, nm, nf))

'''Gradient function. f'. '''
def CFCostGrad(params, myR, myY, mynu, mynm, mynf, myLambda = 0):
    '''
    args:
        params: X+Theta (flattened vector)
        myY: Movies VS Users matrix with their ratings.
        myR: Ratings binary matrix 0/1.
        nu, nm, nf: No of user, no of movies, no of features
    '''
    #This is the partial derivative of the cost function with respect
    #to theta and X.
    #Unroll params into Theta and X and proceed with the derivative 
    #calculation.
    myX, myTheta = unflatten(params, mynu, mynm, mynf)
    term1 = myX.dot(myTheta.T)
    term1 = np.multiply(term1, myR)
    term1 = term1 - myY
    X_grad = term1.dot(myTheta)
    theta_grad = term1.T.dot(myX)

    #Regularization
    X_grad += (myLambda * myX)
    theta_grad += (myLambda * myTheta)
    return flatten_mat(X_grad, theta_grad)

In [ ]:
def check_gradient(params, myR, myY, mynu, mynm, mynf, myLambda = 0):
    epsilon = 0.0001
    eps_vec = np.zeros(len(params))
    myX, myTheta = unflatten(params, mynu, mynm, mynf)
    grads = CFCostGrad(params, myR, myY, mynu, mynm, mynf, myLambda = 1.5)

    for _ in range(10):
        idx = np.random.randint(len(params))
        eps_vec[idx] = epsilon
        loss1 = CFCost(params-eps_vec, myY, myR, mynu, mynm, mynf, myLambda = 1.5)
        loss2 = CFCost(params+eps_vec, myY, myR, mynu, mynm, mynf, myLambda = 1.5)
        
        mygrad = (loss2 - loss1) / (2*epsilon)
        eps_vec[idx] = 0
        print(mygrad, grads[idx], mygrad - grads[idx])


In [ ]:
print("Checking gradient function for lambda = 0:")
check_gradient(flatten_mat(X, Theta), ratings, Y, nu, nm, nf, myLambda = 0)
print("\nChecking gradient function for lambda = 1.5:")
check_gradient(flatten_mat(X, Theta), ratings, Y, nu, nm, nf, myLambda = 1.5)

### - Learn parameters
Add own ratings to the data to be trained on and perform minimzation/gradient descent on params(Theta+X)

In [ ]:
"""Learn parameters for recommendation"""
#Add a vector for my own recommendations. 
my_ratings = np.zeros([nm,1])
my_ratings[0]   = 4
my_ratings[97]  = 2
my_ratings[6]   = 3
my_ratings[11]  = 5
my_ratings[53]  = 4
my_ratings[63]  = 5
my_ratings[65]  = 3
my_ratings[68]  = 5
my_ratings[182] = 4
my_ratings[225] = 5
my_ratings[354] = 5

#Current nm, nu, nf are:
print("Old nm, nu, nf", nm, nu, nf)

#Add my ratings vector to the matrices. 
Y = np.hstack((Y, my_ratings))
myR = my_ratings > 0
ratings = np.hstack((ratings, myR.astype(int)))
print(Y.shape, ratings.shape)

In [ ]:
nm, nu = Y.shape
X, Theta = np.random.rand(nm, nf), np.random.rand(nu, nf)
my_flat_mat = flatten_mat(X, Theta)
print(my_flat_mat.shape)

In [ ]:
#Call minimizer.
mylambda = 10.
res = fmin_cg(CFCost, x0 = my_flat_mat, \
              fprime = CFCostGrad, args = (ratings, Y, nu, nm, nf, mylambda), \
              maxiter=50, disp = True, full_output = True)
print("Prediction Matrix is:", res[0], ", with shape:", res[0].shape)

In [ ]:
print(nu, nm, nf)
new_X, new_theta = unflatten(res[0], nu, nm, nf)
pred_mat = new_X.dot(new_theta.T)

def normalizeRatings(myY, myR):
    # The mean is only counting movies that were rated
    Ymean = np.sum(myY,axis=1)/np.sum(myR,axis=1)
    Ymean = Ymean.reshape((Ymean.shape[0],1))
    
    return myY-Ymean, Ymean 

Ynorm, Ymean = normalizeRatings(Y, ratings)
my_pred = pred_mat[:,-1] + Ymean.flatten()

In [ ]:
#Read movie list. 
movies = []
with open("./aux_data/movie_ids.txt") as f:
    for item in f.read().split("\n"):
        movies.append(" ".join(item.split(" ")[1:]))

#Reverse sort my predictions and get the indexes. Then, get top 10 movies with those indexes.
pred_idxs_sorted = np.argsort(my_pred)
pred_idxs_sorted[:] = pred_idxs_sorted[::-1]


print("\033[1;34mTOP RECOMMENDATIONS FOR YOU:\033[1;m")
for i in range(10):
    print('\033[1;33mPredicting rating for movie\033[1;m', \
          str(my_pred[pred_idxs_sorted[i]])[:4],movies[pred_idxs_sorted[i]])

my_ratings = my_ratings.flatten()
print("\n\033[1;34mORIGINAL RATINGS BY YOU:\033[1;m")
for i in range(len(my_ratings)):
    if my_ratings[i] > 0:
        print('\033[1;33mRated for movie\033[1;m', my_ratings[i], movies[i])